In [32]:
import json
import sys
sys.path.append("..")
import tensorflow as tf
from pathlib import Path
from src.dnn.loaders.VARdataloader import DataLoaderVAR as DataLoader

PATH_EXP = "/data/bacteria/experiments/autoencoders/6mer/26122023-2"

In [35]:
# encoder = tf.keras.models.load_model('/data/bacteria/experiments/autoencoders/6mer/26122023-2/models/encoder.keras')
encoder =tf.keras.models.load_model(f"{PATH_EXP}/models/encoder.keras")
# Show the model architecture
encoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_encoder (InputLayer)  [(None, 64, 64, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 16)        160       
                                                                 
 batch_normalization (Batch  (None, 64, 64, 16)        64        
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        4640      
                                                                 
 batch_normalization_1 (Bat  (None, 64, 64, 32)        128       
 chNormalization)                                                
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 64)        1849

In [36]:
with open(f"{PATH_EXP}/split-train-val-test.json","r") as fp:
    datasets = json.load(fp)

In [37]:
list_files = datasets["id_labels"]["test"][:21]

In [38]:
# preprocessing of each FCGR to feed the model 
preprocessing = lambda x: x / x.max() 

# compute embeddings
trainval_data = DataLoader(
    list_paths=list_files,
    batch_size=10,
    shuffle=False,
    preprocessing=preprocessing,
    inference_mode=True
)

# embeddings train+val
embeddings = []
for data in iter(trainval_data):
    encoded_imgs = encoder(data).numpy()
    embeddings.append(encoded_imgs)

In [39]:
import numpy as np 
all_emb = np.concatenate(embeddings, axis=0)

In [40]:
len(all_emb)

21

In [41]:
all_emb

array([[ 97.584114,   0.      ,   0.      , ...,   0.      , 486.8559  ,
          0.      ],
       [843.95386 , 723.7399  ,   0.      , ..., 183.53992 ,   0.      ,
        142.3568  ],
       [799.2456  , 286.15045 ,   0.      , ...,   0.      , 684.40643 ,
        323.4834  ],
       ...,
       [849.60333 , 289.29495 ,   0.      , ...,   0.      , 674.723   ,
        432.21732 ],
       [  0.      , 359.7828  ,   0.      , ...,   0.      , 564.69556 ,
          0.      ],
       [772.81055 , 383.15405 ,   0.      , ...,   0.      , 483.15747 ,
        407.27057 ]], dtype=float32)